# Benchmark results aggregator

This notebook helps to aggregate the benchmark results collected from a DynamoDB table.

In [1]:
%%capture --no-display
%load_ext autoreload
%autoreload 2

%pip install -q boto3 numpy pandas python-dotenv openpyxl

### Initialization (imports and constants)

In [2]:
import os

from dotenv import load_dotenv

# Define the environment variables below in a ".env" file: `load_dotenv()`
# will source them automatically.
load_dotenv()

# AWS region and table name for where the benchmark results are stored.
REGION = os.environ.get("DYNAMODB_REGION")
TABLE = os.environ.get("DYNAMODB_TABLE")

# S3 Connector for PyTorch versions to query, to compare benchmark results.
PREVIOUS_VERSION = "1.2.7"
NEXT_VERSION = "1.3.0"

### Functions

In [3]:
from datetime import datetime
from typing import List

import numpy as np
import boto3


def query_dynamodb(
    region: str, table_name: str, old_version: str, new_version: str
) -> List[dict]:
    """Query DynamoDB for the latest run results."""
    dynamodb = boto3.resource("dynamodb", region_name=region)

    statement = f'SELECT * FROM "{table_name}" WHERE s3torchconnector_version IN [?, ?]'
    params = [old_version, new_version]
    response = dynamodb.meta.client.execute_statement(
        Statement=statement, Parameters=params
    )

    return response["Items"]


def transform(run_results: List[dict]) -> List[dict]:
    """Build a list of row to be later concatenated in a :class:`pd.DataFrame`."""
    rows = []
    for run_result in run_results:
        for job_result in run_result["job_results"]:
            metrics_averaged = {
                k: float(np.mean(v))  # `float()` to cast away the `Decimal` part
                for k, v in job_result["metrics"].items()
                if k != "utilization"
            }
            row = {
                "version": run_result["s3torchconnector_version"],
                "scenario": run_result["scenario"],
                "disambiguator": run_result.get("disambiguator"),
                "timestamp_utc": datetime.fromtimestamp(
                    float(run_result["timestamp_utc"])
                ),
                **metrics_averaged,
                "config": job_result["config"],
                "ec2_metadata": run_result["ec2_metadata"],
            }
            rows.append(row)

    return rows

### Exploit data

In [4]:
_run_results = query_dynamodb(REGION, TABLE, PREVIOUS_VERSION, NEXT_VERSION)

In [ ]:
import pandas as pd

_table = pd.DataFrame()

if _run_results:
    _data = transform(_run_results)
    _table = pd.json_normalize(_data).set_index("version")

_table

In [ ]:
import string
import random

_suffix = "".join(random.choices(string.ascii_letters, k=5))
_filename = f"benchmark_results_{_suffix}.csv"

if not _table.empty:
    _table.to_csv(_filename)
    print(f"CSV written to {_filename}")